In [ ]:
! pip install opencv-python

In [ ]:
import os
import cv2 
import csv

# xml 파일 파싱 라이브러리 
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

In [ ]:
# xml 파일 불러오기

fileDir = "C://Users//김영민//Downloads//이상행동 CCTV 영상" # 데이터셋 저장 경로
fileExt = ".xml" # 확장자명

frame = []
count = 0 # count 번째 사진 

for category in os.listdir(fileDir) : # 이상행위 별 폴더
    for split in os.listdir(fileDir + "//" + category) : # 해당 이상행위 영상 분할압축 폴더 중 다운로드 받은 파일 폴더
        for folder in os.listdir(fileDir + "//" + category + "//" + split) :
            for file in os.listdir(fileDir + "//" + category + "//" + split + "//" + folder) :
                if file.endswith(fileExt) : # 파일명에 .xml 이 포함된 경우 
                    xml_file = fileDir + "//" + category + "//" + split + "//" + folder + "//" + file
                    doc = ET.parse(xml_file)
                    root = doc.getroot() # root 노드 가지고 오기
                    
                    # xml 파일에서 시작/끝 프레임 수 추출
                    for object in root.iter("action"):
                        start = int(object.find("frame").findtext("start"))
                        end = int(object.find("frame").findtext("end"))

                        framelist = [start, end]
                        frame.append(framelist)
                    
                    for i in range(len(frame)) :
                        filename = file[:-4]
                        print(filename)

                        video = fileDir + "//" + category + "//" + split + "//" + folder + "//" + filename + ".mp4"
                        savepath = "C://cap//" + category[0:2] # 경로에 한글 포함되면 안됨
                        
                        # 기존에 frame dir가 없는 경우, 해당 dir를 생성한다.
                        if not os.path.exists(savepath):
                            os.makedirs(savepath)

                        vidcap = cv2.VideoCapture(video)

                        fps = vidcap.get(cv2.CAP_PROP_FPS)
                        frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT) 
                        duration = frame_count / fps 

                        increase_width = 10 # 스크린샷 캡처 간격 설정
                        start_frame = frame[i][0] 
                        success = True
                        
                        while success :
                            vidcap.set(1, start_frame)
                            success,image = vidcap.read() 
                            image = cv2.resize(image, (960, 540))
                            print(start_frame, "번째 프레임 캡쳐") 
                            cv2.imwrite(savepath + "//%d.jpg"%count, image) # 저장시킬 위치 주소 
                            print("saved image %d.jpg" % count)
                            count += 1
                            start_frame += increase_width 
                            if start_frame > frame[i][1] :
                                break 
                    
                    frame = []
                    print('----- Finish Video Capture! -----')

